In [1]:
!pip install -U scikit-learn
!pip install -U torch

In [2]:
#files
import sys
#from os import listdir
sys.path.append('../data')
sys.path.append('../Librerias')
#from os.path import isfile, join

#own libs
import utils
import caract as dc
import dataset as ds
from models import SimpleRNN, LSTM
#from lstm_models import LSTM
import utils_2 as ut2
from traindata import trainData
import errores as er



#data and visualization
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, root_mean_squared_error, r2_score
from sklearn.svm import SVR
import pickle

sns.set_style("darkgrid")


import numpy as np
from tqdm import tqdm



np.random.seed(seed=77)

In [3]:
#fpath = "/Users/consu/OneDrive/Documentos/GitHub/Tesis/data/low/"
fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/low/"
fname = "file1_080124.mat"
#cutoff = [8/1000, 11/1000]

data = trainData(fpath, fname, 'Amplitude')
train, val, test = data.split_data('Amplitude')


train = utils.subsample(train, 2)
val = utils.subsample(val, 2)
test = utils.subsample(test, 2)

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train.reshape(-1, 1))
val_scaled = scaler.transform(val.reshape(-1, 1))
test_scaled = scaler.transform(test.reshape(-1, 1))

x_train, y_train = ut2.create_sequences(train_scaled[:10000], 1000, 1)
x_train = x_train.squeeze()
y_train = y_train.squeeze()

x_val, y_val = ut2.create_sequences(val_scaled[:5000], 1000, 1)
x_val = x_val.squeeze()
y_val = y_val.squeeze()

x_test, y_test = ut2.create_sequences(test_scaled, 1000, 1)
x_test = x_test.squeeze()
y_test = y_test.squeeze()

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

0         0.001705
1         0.001436
2         0.001494
3         0.001446
4         0.001489
            ...   
599995    0.000679
599996    0.000686
599997    0.000843
599998    0.000868
599999    0.001361
Name: Amplitude, Length: 600000, dtype: float64
0         0.001705
1         0.001436
2         0.001494
3         0.001446
4         0.001489
            ...   
599995    0.000679
599996    0.000686
599997    0.000843
599998    0.000868
599999    0.001361
Name: Amplitude, Length: 600000, dtype: float64
0         0.001705
1         0.001436
2         0.001494
3         0.001446
4         0.001489
            ...   
599995    0.000679
599996    0.000686
599997    0.000843
599998    0.000868
599999    0.001361
Name: Amplitude, Length: 600000, dtype: float64
0         0.001705
1         0.001436
2         0.001494
3         0.001446
4         0.001489
            ...   
599995    0.000679
599996    0.000686
599997    0.000843
599998    0.000868
599999    0.001361
Name: Amplitude, Len

((9000, 1000), (9000,), (4000, 1000), (4000,), (44000, 1000), (44000,))

In [4]:
filename = 'SVR_model.sav'
model = pickle.load(open(filename, 'rb'))


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVR from version 1.0.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## **predicciones a 50 steps**

In [38]:
steps = 50
x_test , y_test = ut2.create_sequences(test, 1000, steps)

In [39]:
test50 = ut2.rollingWindowPrediction_SVR(model, x_test, steps)

100%|██████████| 43951/43951 [08:12<00:00, 89.32it/s]


In [40]:
np.array(test50).shape, y_test.shape

((43951, 50), (43951, 50))

In [36]:
mse, mape, r2, rmse = [], [], [], []

for i in tqdm(range(len(y_test))):
    mse.append(mean_squared_error(y_test[i], test50[i]))
    mape.append(mean_absolute_percentage_error(y_test[i], test50[i]))
    r2.append(r2_score(y_test[i], test50[i]))
    rmse.append(root_mean_squared_error(y_test[i], test50[i]))

100%|██████████| 951/951 [00:01<00:00, 497.51it/s]


In [37]:
print('Media MSE:\t', np.mean(mse))
print('Media MAPE:\t', np.mean(mape))
print('Media R2:\t', np.mean(r2))
print('Media RMSE:\t', np.mean(rmse))


Media MSE:	 0.01885037889973827
Media MAPE:	 166.99432824296755
Media R2:	 -4311062.478650903
Media RMSE:	 0.1372965574403094


In [41]:
mse, mape, r2, rmse = er.calculate_errors(test50, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.018840635153868582
Mean Absolute Percentage Error: 195.18356863087757
R2 Score: -53048.039327155064
Root Mean Squared Error: 0.12570993003393272


## **predicciones a 100 steps**

In [42]:
steps = 100
x_test , y_test = ut2.create_sequences(test, 1000, steps)

In [43]:
test100 = ut2.rollingWindowPrediction_SVR(model, x_test, steps)

100%|██████████| 43901/43901 [16:23<00:00, 44.64it/s]


In [44]:
mse, mape, r2, rmse = er.calculate_errors(test100, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.042971675370006075
Mean Absolute Percentage Error: 297.97884635360276
R2 Score: -120888.88922940672
Root Mean Squared Error: 0.19191000283741486


## **Predicciones a 500 steps**

In [45]:
steps = 500
x_test , y_test = ut2.create_sequences(test, 1000, steps)

In [46]:
test500 = ut2.rollingWindowPrediction_SVR(model, x_test, steps)

100%|██████████| 43501/43501 [1:20:18<00:00,  9.03it/s]


In [48]:
mse, mape, r2, rmse = er.calculate_errors(test500, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.12571776494313622
Mean Absolute Percentage Error: 526.5052749421508
R2 Score: -351885.060076478
Root Mean Squared Error: 0.3388772060139706


## **Predicciones a 1000 steps**

In [49]:
steps = 1000
x_test , y_test = ut2.create_sequences(test, 1000, steps)
test1000 = ut2.rollingWindowPrediction_SVR(model, x_test, steps)

100%|██████████| 43001/43001 [2:35:19<00:00,  4.61it/s]  


In [50]:
mse, mape, r2, rmse = er.calculate_errors(test1000, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.09679396681522627
Mean Absolute Percentage Error: 459.67499731735523
R2 Score: -269900.1500708661
Root Mean Squared Error: 0.2959685853147217
